## Setup

In [ ]:
!pip install pyproj

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.7 MB/s eta 0:00:00


In [ ]:
import requests
import urllib.parse
import pandas as pd
import json

from pyproj import CRS, Transformer
from shapely.geometry import Point
from shapely.ops import transform

from datetime import datetime, timedelta
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
original_dataset_path = 'nasa_glc_india.csv'
cleaned_dataset_path = 'nasa_glc_india_cleaned.csv'
cleaned_dataset_with_rainfall_path = 'nasa_glc_india_cleaned_rainfall.csv'
rainfall_data_path = 'landslide_rainfall_data.json'
indexed_coord_path = '/content/drive/MyDrive/indexed_coordinates.json'

## Data Cleaning

In [ ]:
df = pd.read_csv(original_dataset_path)

In [ ]:
"""
Useful headers: location_accuracy, landslide_category (?), landslide_trigger, longitude, latitude
"""
print(df.landslide_trigger.value_counts(), end="\n\n")
print(df.location_accuracy.value_counts(), end="\n\n")
print(df.landslide_category.value_counts(), end="\n\n")

downpour             726
rain                 407
continuous_rain      285
unknown              148
monsoon               68
construction          36
snowfall_snowmelt     23
mining                17
tropical_cyclone      14
flooding               9
other                  7
leaking_pipe           1
Name: landslide_trigger, dtype: int64

5km        538
1km        341
25km       284
10km       265
50km       178
unknown     81
exact       44
100km        7
250km        2
Name: location_accuracy, dtype: int64

landslide              1500
mudslide                106
rock_fall                84
debris_flow              17
complex                  15
other                     6
unknown                   4
translational_slide       3
riverbank_collapse        2
snow_avalanche            2
rotational_slide          1
creep                     1
Name: landslide_category, dtype: int64



In [ ]:
rainfall_landslide_triggers = [
    'downpour', 'rain', 'continuous_rain',  
    'monsoon', 'tropical_cyclone', 'flooding'
]
non_rainfall_landslide_triggers = [
    'construction', 'snowfall_snowmelt', 'mining', 'leaking_pipe', 'other'
]

In [ ]:
"""
Remove landslides on the basis of landslide_trigger (triggering factor) and location_accuracy
"""
df_cleaned = df[
    (
        (df['landslide_trigger'] == 'downpour') | 
        (df['landslide_trigger'] == 'rain') |
        (df['landslide_trigger'] == 'continuous_rain') | 
        (df['landslide_trigger'] == 'monsoon') | 
        (df['landslide_trigger'] == 'tropical_cyclone') |
        (df['landslide_trigger'] == 'flooding')
    ) & 
    (
        (df['location_accuracy'] == 'exact') | 
        (df['location_accuracy'] == '1km') | 
        (df['location_accuracy'] == '5km') | 
        (df['location_accuracy'] == '10km')
    )
]
print(df_cleaned.landslide_trigger.value_counts(), end="\n\n")
print(df_cleaned.location_accuracy.value_counts(), end="\n\n")
print(df_cleaned.landslide_category.value_counts(), end="\n\n")

downpour            472
rain                245
continuous_rain     219
monsoon              57
tropical_cyclone      9
flooding              7
Name: landslide_trigger, dtype: int64

5km      470
1km      275
10km     228
exact     36
Name: location_accuracy, dtype: int64

landslide              865
mudslide                63
rock_fall               54
debris_flow             12
complex                  6
other                    4
translational_slide      2
riverbank_collapse       1
rotational_slide         1
unknown                  1
Name: landslide_category, dtype: int64



In [ ]:
df_cleaned.to_csv(cleaned_dataset_path, index=False)

## Data Integration

### Rainfall

In [ ]:
"""
Rainfall:
1. Short term rainfall (i.e. Sum of rain on the day of the landslide and also a day prior) 
2. Long term rainfall (i.e. Sum of rain on days ranging from 2 to 10 days prior landslide)
https://archive-api.open-meteo.com/v1/archive?latitude=52.52&longitude=13.41&start_date=2023-02-19&end_date=2023-03-20&daily=rain_sum&timezone=Asia%2FKolkata
"""
df_cleaned = pd.read_csv(cleaned_dataset_path)

if 'short_term_rainfall' not in df_cleaned:
  df_cleaned['short_term_rainfall'] = ''
if 'long_term_rainfall' not in df_cleaned:
  df_cleaned['long_term_rainfall'] = ''

In [ ]:
date_format = '%Y-%m-%d'

def get_rainfall(lat, lon, start_date, end_date):
  url = 'https://archive-api.open-meteo.com/v1/archive'
  params = {
      'latitude': lat,
      'longitude': lon,
      'start_date': start_date.strftime(date_format),
      'end_date': end_date.strftime(date_format),
      'daily': 'rain_sum',
      'timezone': 'Asia/Kolkata'
  }
  return requests.get(url + '?' + urllib.parse.urlencode(params)).json()

In [ ]:
for i in range(len(df_cleaned['event_date'])):
  if df_cleaned['short_term_rainfall'][i] == '':
    date_only = df_cleaned['event_date'][i].strip().split()[0].strip()
    
    end_date = datetime.strptime(date_only, date_format)
    start_date = end_date - timedelta(days=10)

    # No need to ping the server for data, use 'landslide_rainfall_data.json'
    data = get_rainfall(df_cleaned['latitude'][i], df_cleaned['longitude'][i], start_date, end_date)

    df_cleaned['short_term_rainfall'][i] = sum(data['daily']['rain_sum'][-2:])
    df_cleaned['long_term_rainfall'][i] = sum(data['daily']['rain_sum'][:-2])

In [ ]:
df_cleaned.to_csv(cleaned_dataset_with_rainfall_path, index=False)

### Elevation

In [ ]:
"""
Elevation:
1. Elevation relief (i.e. Difference between the maximum and minimum elevation within the landslide confidence area)
https://www.calcmaps.com/ajax.php?op=elevation_s1&lat=30.477367286124487&lng=77.65326695265027&_=1678902713402
"""
df_rainfall = pd.read_csv(cleaned_dataset_with_rainfall_path)

length_transform = { 'exact': 0.03, '1km': 1.0, '5km': 5.0, '10km': 10.0 }

if 'elevation_relief' not in df_rainfall:
  df_rainfall['elevation_relief'] = ''

In [ ]:
def geodesic_point_buffer(lat, lon, meters, res):
    # Azimuthal equidistant projection
    aeqd_proj = CRS.from_proj4(
        f"+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0")
    tfmr = Transformer.from_proj(aeqd_proj, aeqd_proj.geodetic_crs)
    buf = Point(0, 0).buffer(distance=meters, resolution=res)
    return transform(tfmr.transform, buf).exterior.coords[:]

In [ ]:
coords_map = {}

In [ ]:
for i in range(len(df_rainfall['latitude'])):
  lat = df_rainfall['latitude'][i]
  lon = df_rainfall['longitude'][i]
  radius_meters = int(length_transform[df_rainfall['location_accuracy'][i]] * 1000)
  
  coords_within_circle = set([])
  for j in range(30, radius_meters + 1, 30):
    coords = set(geodesic_point_buffer(lat, lon,  j, (240 * j / 10000)))
    coords_within_circle = coords_within_circle.union(coords)
  
  print(i, len(coords_within_circle))
  coords_map[i] = list(coords_within_circle)

0 159524
1 159524
2 1556
3 1556
4 159524
5 159524
6 39604
7 1556
8 1556
9 1556
10 159524
11 39604
12 39604
13 1556
14 39604
15 159524
16 39604
17 1556
18 1556
19 4
20 1556
21 1556
22 159524
23 39604
24 39604
25 39604
26 39604
27 39604
28 39604
29 39604
30 159524


KeyboardInterrupt: ignored

In [ ]:
coords_map

In [ ]:
def send_request(coords):
  locations = "|".join([f"{lat},{lon}" for lon, lat in coords])
  r = requests.get(url="https://api.opentopodata.org/v1/srtm30m", params={'locations': locations})
  return r.json() if r.status_code == 200 else {}